In [72]:
import pandas as pd
import numpy as np

In [73]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,0,100.0,0.841611,10.0,4.8,20.612526,11.08810,2.766,1.732,0.860,0.496070,0.91457,6.0
1,1,100.0,7.558488,10.0,4.8,20.298893,12.04083,2.755,1.631,0.910,0.492719,0.71760,6.5
2,2,76.0,8.885992,15.6,5.6,33.739258,12.08630,2.828,1.788,0.864,0.481478,1.50633,2.5
3,3,100.0,8.795296,10.0,4.8,20.213349,10.94850,2.648,1.626,0.936,0.489272,0.78937,6.0
4,4,116.0,9.577996,11.6,4.8,24.988133,11.82448,2.766,1.682,0.896,0.492736,1.86481,6.0


In [74]:
sub_df = pd.read_csv('sample_submission.csv')

In [75]:
train_df.shape

(10407, 13)

In [76]:
test_df = pd.read_csv('test.csv')
test_df.head()
submission_id = test_df.id

In [77]:
train_df = train_df.drop(['id'], axis = True)
test_df = test_df.drop(['id'], axis=True)

In [78]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy, MeanAbsoluteError

In [79]:
X = train_df.drop(['Hardness'], axis=True)
y = train_df['Hardness']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [80]:
def scale_dataset(X_train, X_test):
    standard_scaler = StandardScaler()
    x_train_scaled = pd.DataFrame(
    standard_scaler.fit_transform(X_train),
    columns=X_train.columns
    )
    x_test_scaled = pd.DataFrame(
    standard_scaler.transform(X_test),
    columns=X_test.columns
    )
    return x_train_scaled, x_test_scaled

x_train_scaled, x_test_scaled = scale_dataset(X_train, X_test)

In [81]:
hidden_unit1 = 128
hidden_unit2 = 512
hidden_unit3 = 512
hidden_unit4 = 256
learning_rate = 0.01

def build_model_using_sequential():
    model = Sequential([
        Dense(hidden_unit1, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_unit2, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_unit3, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_unit4, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(1, kernel_initializer='normal')
    ])
    return model

model = build_model_using_sequential()

In [82]:
mse = MeanAbsoluteError()
model.compile(loss=MeanAbsoluteError(),optimizer=Adam(learning_rate=learning_rate), metrics=[mse])
history = model.fit(
    x_train_scaled.values,
    y_train.values,
    epochs=50,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/50
105/105 [==============================] - 1s 7ms/step - loss: 1.4500 - mean_absolute_error: 1.4488 - val_loss: 1.5025 - val_mean_absolute_error: 1.5979
Epoch 2/50
105/105 [==============================] - 1s 6ms/step - loss: 1.1725 - mean_absolute_error: 1.1698 - val_loss: 1.0385 - val_mean_absolute_error: 1.1218
Epoch 3/50
105/105 [==============================] - 1s 6ms/step - loss: 1.1467 - mean_absolute_error: 1.1488 - val_loss: 0.9933 - val_mean_absolute_error: 1.0711
Epoch 4/50
105/105 [==============================] - 1s 6ms/step - loss: 1.1603 - mean_absolute_error: 1.1599 - val_loss: 1.0443 - val_mean_absolute_error: 1.1289
Epoch 5/50
105/105 [==============================] - 1s 6ms/step - loss: 1.0947 - mean_absolute_error: 1.0925 - val_loss: 1.1675 - val_mean_absolute_error: 1.2174
Epoch 6/50
105/105 [==============================] - 1s 6ms/step - loss: 1.0995 - mean_absolute_error: 1.1003 - val_loss: 1.1012 - val_mean_absolute_error: 1.1571
Epoch 7/50
105/1

In [83]:
def dataset_scaled(test_df):
    standard_scaler = StandardScaler()
    x_pred_scaled = pd.DataFrame(
    standard_scaler.fit_transform(test_df),
    columns=test_df.columns
    )
    
    return x_pred_scaled

In [84]:
x_scales = dataset_scaled(test_df)

In [85]:
y_pred = model.predict(x_scales)

217/217 [==============================] - 0s 2ms/step


In [86]:
y_pred

array([[2.933456 ],
       [2.933456 ],
       [5.7486625],
       ...,
       [5.7486625],
       [2.933456 ],
       [2.933456 ]], dtype=float32)

In [88]:
sub_df['Hardness'] = y_pred

In [89]:
sub_df.to_csv('submission.csv', index=False)

In [90]:
df = pd.read_csv('submission.csv')

In [91]:
df.shape

(6939, 2)

In [93]:
df['Hardness'].dtype

dtype('float64')